# Elastic, Viscoplastic model with linear hardening

FramCos paper preparation

In [ ]:
%matplotlib widget
from bmcs_matmod.api import GSMRM
import matplotlib.pylab as plt 
import sympy as sp
import numpy as np
from bmcs_matmod.gsm.potentials.potential1d_t_e_vp_d_likh import Potential1D_T_E_VP_D_LIKH_SymbExpr 
sp.init_printing()

## Helmholtz free energy

In [ ]:
p1d = Potential1D_T_E_VP_D_LIKH_SymbExpr()
p1d.F_

## Gibbs free energy

In [ ]:
eps_a = p1d.eps_a
sig_a = p1d.sig_a
dF_du = p1d.F_.diff(eps_a)
# dF_du = dF_du.xreplace({h: 0 for h in dF_du.atoms(sp.DiracDelta)})
# dF_du = dF_du.xreplace({h: 1 for h in dF_du.atoms(sp.Heaviside)})
u_sig_ = sp.Matrix([ sp.solve(sp.Eq(sig_i, dF_du_i), u_i)[0] for sig_i, u_i, dF_du_i in 
                            zip(sig_a, eps_a, dF_du)])
subs_u_sig_ = dict(zip(eps_a, u_sig_))
sp.simplify(dF_du), subs_u_sig_

In [ ]:
sig_x_u_ = (sig_a.T * eps_a)[0]
#G_expr = sig_x_u_ - p1d.F_
G_expr = p1d.F_ - sig_x_u_
G_ = sp.simplify(G_expr.subs(subs_u_sig_))
G_

## GSM drivers

In [ ]:
gsm_F = GSMRM(
    name = 'gsm_F_1d_t_e_vp_likh',
    u_vars = p1d.eps_a,
    sig_vars = p1d.sig_a,
    T_var = p1d.T,
    m_params = p1d.mparams,
    Eps_vars = p1d.Eps_vars,
    Sig_vars = p1d.Sig_vars,
    Sig_signs = p1d.Sig_signs,
    F_expr = p1d.F_,
    f_expr = p1d.f_,
    phi_ext_expr = p1d.phi_ext_,
    t_relax = p1d.t_relax_,
    m_param_codenames =  p1d.m_param_codenames
)

gsm_G = GSMRM(
    name = 'gsm_G_1d_t_e_vp_d',
    u_vars = p1d.sig_a,
    sig_vars = p1d.eps_a,
    T_var = p1d.T,
    m_params = p1d.mparams,
    Eps_vars = p1d.Eps_vars,
    Sig_signs = p1d.Sig_signs,
    Sig_vars = p1d.Sig_vars,
    #Sig_signs = (1, -1, -1, 1),
    F_expr = G_,
    #dF_sign = -1,
    f_expr = p1d.f_,
    phi_ext_expr = p1d.phi_ext_,
    t_relax = p1d.t_relax_,
    m_param_codenames =  p1d.m_param_codenames
)

In [ ]:
sp.simplify(p1d.f_)

In [ ]:
gsm_F.df_dSig_ - gsm_G.df_dSig_

In [ ]:
gsm_F.f_.diff(gsm_F.Eps.as_explicit()) - gsm_G.f_.diff(gsm_G.Eps.as_explicit())

In [ ]:
gsm_F.Phi_ - gsm_G.Phi_

In [ ]:
gsm_F.Sig_, gsm_G.Sig_

In [ ]:
gsm_F.dSig_dEps_, sp.factor(gsm_G.dSig_dEps_)

In [ ]:
gsm_F.save_to_disk()
gsm_G.save_to_disk()

In [ ]:
gsm_F.df_dEps_, gsm_G.df_dEps_

## Monotonic loading

In [ ]:
_f_c = 44
_f_t = -0.1 * _f_c
_X_0 = (_f_c + _f_t) / 2
_f_s = (_f_c - _f_t) / 2
_E = 50000
_KH_factor = 4
_KH = _E * _KH_factor
_K_ratio = 0.8 # 0.01 # 0.015
_K = _KH * _K_ratio
_H = _KH * (1 - _K_ratio)
material_params = dict(
    E=_E, 
    gamma=_H * 0, # _E * 10, 
    X_0=_X_0,  
    K=_K / 10, # _E / 5,
    f_c=_f_s,
    S=1e+6, # 0.0008,
    c=2.3,
    r=1,
    eta=20000,
    T_0=20,
    C_v=1e+6, # 0.01, # 0.0001, 
    d_N=1,
    beta=0, # 0.5,
    z_0 = 0,
    alpha_therm = 0,
)
p1d.mparams

In [ ]:
def gsm_run(gsm_, u_ta, T_t, t_t, **material_params):
    response = gsm_.get_response(u_ta, T_t, t_t, **material_params)
    _t_t, _u_tIa, _T_t, _Eps_tIb, _Sig_tIb, _iter_t, _dF_dEps_t, lam_t, (d_t_t, d_u_ta) = response
    _u_atI, _Eps_btI, _Sig_btI, _dF_dEps_btI = [np.moveaxis(v_, -1, 0) for v_ in (_u_tIa, _Eps_tIb, _Sig_tIb, _dF_dEps_t)]
    _sig_atI = gsm_.get_sig(_u_atI, _T_t, _Eps_btI, _Sig_btI, **material_params )
    return _t_t, _u_atI, _sig_atI, _T_t, _Eps_btI, _Sig_btI, _dF_dEps_btI, lam_t, (d_t_t, d_u_ta)

gsm_F.vp_on = True
gsm_F.update_at_k = False

def get_cyclic_load(max_s, max_t, n_cycles, n_incr):
    # Generating loading history
    s_arr = np.tile(np.array([-1, 1]), n_cycles) * np.linspace(0, max_s, 2 * n_cycles)
    s_arr = np.interp(np.linspace(0, max_t, n_incr * len(s_arr)), np.linspace(0, max_t, len(s_arr)), s_arr)

    # time array as input
    t_arr = np.linspace(0, max_t, len(s_arr))
    return s_arr, t_arr

In [ ]:
material_params

In [ ]:
gsm_F.m_param_codenames

In [ ]:
gsm_F.param_codenames

In [ ]:
response_values = {}
max_s = 0.01
dot_s_list = [0.00005, 0.0005, 0.005]
for dot_s in dot_s_list:
    print('dot_s', dot_s)
    u_t, t_t = get_cyclic_load( n_cycles=1, max_s=max_s, 
                                  max_t=max_s/dot_s, n_incr = 10)
    response_values[dot_s] = gsm_run(gsm_F, u_t[:,np.newaxis], 20 + 0 * t_t, t_t, **material_params)

In [ ]:
from scipy.integrate import cumtrapz

def plot_monotonic(response_values):
    fig, ((ax, ax_T, ax_Diss), (ax_u_p, ax_lam, ax_omega)) = plt.subplots(2,3, figsize=(12,6), tight_layout=True)
    ax_z = ax_lam.twinx()

    for dot_s, resp in response_values.items():
        
        label = f'$\dot{{s}}$ = {dot_s}'
        _t_t, _u_atI, _sig_atI, _T_t, _Eps_btI, _Sig_btI, _dF_dEps_btI, lam_t, (d_t_t, d_u_ta) = resp 
        _u_p_atI, _z_atI, _alpha_atI, _omega_atI = gsm_F.Eps_as_blocks(_Eps_btI)
        _, _Z_atI, _X_atI, _Y_atI = gsm_F.Eps_as_blocks(_Sig_btI)
        ax.plot(_u_atI[0, :, 0], _sig_atI[0, :, 0], label=label);
        ax.legend()
        ax.set_title(r'stress-strain')
        ax.set_ylabel(r'$\varsigma$')
        ax.set_xlabel(r'$\varepsilon$')

        ax_T.plot(_u_atI[0, :, 0], _T_t, label=label);
        ax_T.legend()
        ax_T.set_title(r'temperature')
        ax_T.set_ylabel(r'$\vartheta$')
        ax_T.set_xlabel(r'$\varepsilon$')

        Diss_btI_F = cumtrapz(_dF_dEps_btI, _Eps_btI, initial=0, axis=1)
        ax_Diss.plot(_t_t, np.sum(Diss_btI_F[...,0], axis=0), alpha=1, label='F')
        ax_Diss.set_title(r'dissipation')

        r = material_params['r']
        c = material_params['c']
        ax_u_p.plot(_u_atI[0, :, 0], (1 - _omega_atI[0, :, 0]**c)* (_Y_atI[0, :, 0] / material_params['S'])**r)
        ax_u_p.set_xlabel(r'$\varepsilon$/-')
        ax_u_p.set_ylabel(r'$\varepsilon_\mathrm{p}$/-')
        ax_u_p.set_title(r'$\partial{\varphi}/{\partial Y} =  (1-\omega)^c (Y/S)^r$')

        ax_lam.plot(_u_atI[0, :, 0], lam_t)
        ax_lam.set_xlabel(r'$\varepsilon$/-')
        ax_lam.set_ylabel(r'$\lambda$/-')
        ax_lam.set_title(r'plastic multiplier: $\dot{\lambda}$')

        ax_z.plot(_u_atI[0, :, 0], _Z_atI[0, :, 0], ls='dashed')
        ax_z.set_ylabel(r'$z$/-')

        ax_omega.plot(_u_atI[0, :, 0], _omega_atI[0, :, 0])
        ax_omega.set_xlabel(r'$\varepsilon$/-')
        ax_omega.set_ylabel(r'$\omega$/-')
        ax_omega.set_title(r'damage: $\omega =  \dot{\lambda} \cdot \partial\varphi / \partial Y$')
    return fig

In [ ]:
plot_monotonic(response_values);

In [ ]:
max_s = 0.003
u_t_F, t_t_F = get_cyclic_load( n_cycles=1, max_s=max_s, 
                                max_t=max_s/dot_s_list[-1], n_incr = 3)
T_t_F = 20 + t_t_F * 0

response_values_F = gsm_run(gsm_F, u_t_F[:,np.newaxis], T_t_F, t_t_F, **material_params)

In [ ]:
_t_t_F, _u_atI_F, _sig_atI_F, _T_t_F, _Eps_btI_F, _Sig_btI_F, _dF_dEps_btI_F, lam_t_F, (d_t_t, d_u_ta) = response_values_F 
argmax_sig_ = np.argmax(_sig_atI_F)
t_t_G = _t_t_F[:argmax_sig_+1]
u_ta_G = _sig_atI_F[0, :argmax_sig_+1, 0].reshape(-1, 1)
T_t_G = 20 + t_t_G * 0
response_values_G = gsm_run(gsm_G, u_ta_G, T_t_G, t_t_G, **material_params)


In [ ]:
fig, ((ax_eps, ax_sig), (ax_lam, ax_z), (ax_eps_p, ax_omega), (ax_d_t, ax_Y)) = plt.subplots(4,2, figsize=(12,10), tight_layout=True)

_t_t_F, _u_atI_F, _sig_atI_F, _T_t_F, _Eps_btI_F, _Sig_btI_F, _dF_dEps_btI_F, _lam_t_F, (_d_t_F, _d_u_ta_F) = response_values_F 
_u_p_atI_F, _z_atI_F, _alpha_atI_F, _omega_atI_F = gsm_F.Eps_as_blocks(_Eps_btI_F)
_, _Z_atI_F, _X_atI_F, _Y_atI_F = gsm_F.Eps_as_blocks(_Sig_btI_F)

_t_t_G, _sig_atI_G, _u_atI_G, _T_t_G, _Eps_btI_G, _Sig_btI_G, _dF_dEps_btI_G, _lam_t_G, (_d_t_G, _d_u_ta_G) = response_values_G 
_u_p_atI_G, _z_atI_G, _alpha_atI_G, _omega_atI_G = gsm_G.Eps_as_blocks(_Eps_btI_G)
_, _Z_atI_G, _X_atI_G, _Y_atI_G = gsm_F.Eps_as_blocks(_Sig_btI_G)


ax_eps.plot(_t_t_F, _u_atI_F[0, :, 0], 'o-', label='F', color='blue',)
ax_eps.plot(_t_t_G, _u_atI_G[0, :, 0], 'o', label='G', color='red', ls='dashed')
ax_eps.legend()
ax_eps.set_title(r'strain-time')
ax_eps.set_ylabel(r'$\varepsilon$')
ax_eps.set_xlabel(r'$t$')

ax_sig.plot(_t_t_F, _sig_atI_F[0, :, 0], 'o-', label='F', color='blue')
ax_sig.plot(_t_t_G, _sig_atI_G[0, :, 0], 'o', label='G', color='red', ls='dashed')
ax_sig.legend()
ax_sig.set_title(r'stress-time')
ax_sig.set_ylabel(r'$\sigma$')
ax_sig.set_xlabel(r'$t$')

ax_lam.plot(_t_t_F, _lam_t_F, 'o-', label='F', color='blue')
ax_lam.plot(_t_t_G, -_lam_t_G, 'o-', label='G', color='red')
ax_lam.legend()
ax_lam.set_title(r'$\lambda$-time')
ax_lam.set_ylabel(r'$\lambda$')
ax_lam.set_xlabel(r'$t$')

ax_z.plot(_t_t_F, _z_atI_F[0, :, 0], 'o-', label='F', color='blue')
ax_z.plot(_t_t_G, _z_atI_G[0, :, 0], 'o-', label='G', color='red')
ax_z.legend()
ax_z.set_title(r'z-time')
ax_z.set_ylabel(r'$z$')
ax_z.set_xlabel(r'$t$')

ax_eps_p.plot(_t_t_F, _u_p_atI_F[0, :, 0], 'o-', label='F', color='blue')
ax_eps_p.plot(_t_t_G, _u_p_atI_G[0, :, 0], 'o-', label='G', color='red')
ax_eps_p.legend()
ax_eps_p.set_title(r'$\varepsilon_\mathrm{p}$-time')
ax_eps_p.set_ylabel(r'$\varepsilon_\mathrm{p}$')
ax_eps_p.set_xlabel(r'$t$')

ax_omega.plot(_t_t_F, _omega_atI_F[0, :, 0], 'o-', label='F', color='blue')
ax_omega.plot(_t_t_G, _omega_atI_G[0, :, 0], 'o-', label='G', color='red')
ax_omega.legend()
ax_omega.set_title(r'$\omega$-time')
ax_omega.set_ylabel(r'$\omega$')
ax_omega.set_xlabel(r'$t$')

r = material_params['r']
c = material_params['c']
ax_Y.plot(t_t_F, (1 - _omega_atI_F[0, :, 0]**c)* (_Y_atI_F[0, :, 0] / material_params['S'])**r, 'o-', label='F', color='blue')
ax_Y.plot(t_t_G, (1 - _omega_atI_G[0, :, 0]**c)* (_Y_atI_G[0, :, 0] / material_params['S'])**r, 'o-', label='F', color='red')
ax_Y.set_xlabel(r'$\varepsilon$/-')
ax_Y.set_ylabel(r'$\varepsilon_\mathrm{p}$/-')
ax_Y.set_title(r'$\partial{\varphi}/{\partial Y} =  (1-\omega)^c (Y/S)^r$')

# ax_d_t.plot(_t_t_F[1:], _d_t_F, 'o-', label='F', color='blue')
# ax_d_t.plot(_t_t_G[1:], _d_t_G, 'o-', label='G', color='red')
ax_d_t.plot(_t_t_F, _Y_atI_F[0, :, 0], 'o-', label='F', color='blue')
ax_d_t.plot(_t_t_G, _Y_atI_G[0, :, 0], 'o-', label='G', color='red')
ax_d_t.legend()
ax_d_t.set_title(r'$Y$-time')
ax_d_t.set_ylabel(r'$Y$')
ax_d_t.set_xlabel(r'$t$')

